In [1]:
# 차량별 데이터 - 완성본
import urllib.request
import json
from bs4 import BeautifulSoup
import pymysql

# URL 설정 (법인, 개인(남성/여성), 국산/수입 구분하여 URL을 설정)
urls = [
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "개인",
        "gender": "남성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "개인",
        "gender": "여성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "개인",
        "gender": "남성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "개인",
        "gender": "여성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "개인",
        "gender": "남성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "개인",
        "gender": "여성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "개인",
        "gender": "남성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "개인",
        "gender": "여성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandTreeMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "수입"
    }
]

# 한국 브랜드 목록
korean_brands = ['기아', '현대', '제네시스', 'KGM', '쉐보레', '르노코리아', '쎄보모빌리티']

# MySQL 데이터베이스 연결
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    database='pjt1',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connection.cursor()

# 테이블 삭제 (옵션: 만약 테이블이 이미 존재한다면)
cursor.execute("DROP TABLE IF EXISTS car_sales")
cursor.execute("DROP TABLE IF EXISTS year_sales")
cursor.execute("DROP TABLE IF EXISTS car_models")
cursor.execute("DROP TABLE IF EXISTS car_brands")

# 테이블 생성 (정규화된 구조로)
create_car_brands_query = """
CREATE TABLE car_brands (
    id INT AUTO_INCREMENT PRIMARY KEY,
    brand_name VARCHAR(255) NOT NULL,
    country VARCHAR(50) NOT NULL -- '국산' 또는 '수입'
)
"""
create_car_models_query = """
CREATE TABLE car_models (
    id INT AUTO_INCREMENT PRIMARY KEY,
    model_name VARCHAR(255) NOT NULL,
    brand_id INT NOT NULL,
    FOREIGN KEY (brand_id) REFERENCES car_brands(id)
)
"""
create_car_sales_query = """
CREATE TABLE car_sales (
    id INT AUTO_INCREMENT PRIMARY KEY,
    model_id INT NOT NULL,
    customer_type ENUM('법인', '개인') NOT NULL,
    gender ENUM('남성', '여성', '해당 없음') NULL,
    year INT NOT NULL,
    count INT NOT NULL,
    FOREIGN KEY (model_id) REFERENCES car_models(id)
)
"""
create_year_sales_query = """
CREATE TABLE year_sales (
    id INT AUTO_INCREMENT PRIMARY KEY,
    model_id INT NOT NULL,
    year INT NOT NULL,
    sales_count INT NOT NULL,
    FOREIGN KEY (model_id) REFERENCES car_models(id)
)
"""

# 테이블 생성 실행
cursor.execute(create_car_brands_query)
cursor.execute(create_car_models_query)
cursor.execute(create_car_sales_query)
cursor.execute(create_year_sales_query)

# 반복문을 통해 각 URL에서 데이터를 가져와 처리
for url_info in urls:
    url = url_info["url"]
    year = url_info["year"]
    customer_type = url_info["customer_type"]
    gender = url_info["gender"]
    country = url_info["country"]

    # HTML 요청 및 파싱
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    # 스크립트 태그에서 데이터를 포함한 텍스트 추출
    li_list = soup.select('body > script')
    text = li_list[5].text

    # JSON 데이터의 시작과 끝 인덱스를 찾아 슬라이싱
    start_index = text.find('{"IMAGE_DATA"')
    end_index = text.rfind(',"result":"OK"}') + len(',"result":"OK"}')
    map_data_text = text[start_index:end_index]

    # 불필요한 슬래시와 따옴표 제거
    map_data_text = map_data_text.replace("\\'", "'")

    # JSON 문자열을 파이썬 딕셔너리로 변환
    data = json.loads(map_data_text)

    for brand, models in data['MAP_DATA'].items():
        # 브랜드가 존재하는지 확인 후 삽입
        cursor.execute("SELECT id FROM car_brands WHERE brand_name = %s AND country = %s", (brand, country))
        brand_row = cursor.fetchone()
        if not brand_row:
            cursor.execute("INSERT INTO car_brands (brand_name, country) VALUES (%s, %s)", (brand, country))
            brand_id = cursor.lastrowid
        else:
            brand_id = brand_row['id']

        for model, count in models.items():
            # 모델이 존재하는지 확인 후 삽입
            cursor.execute("SELECT id FROM car_models WHERE model_name = %s AND brand_id = %s", (model, brand_id))
            model_row = cursor.fetchone()
            if not model_row:
                cursor.execute("INSERT INTO car_models (model_name, brand_id) VALUES (%s, %s)", (model, brand_id))
                model_id = cursor.lastrowid
            else:
                model_id = model_row['id']

            # 판매 데이터 삽입
            cursor.execute("""
                INSERT INTO car_sales (model_id, customer_type, gender, year, count)
                VALUES (%s, %s, %s, %s, %s)
            """, (model_id, customer_type, gender, year, count))

            # 연도별 판매량 삽입
            cursor.execute("""
                INSERT INTO year_sales (model_id, year, sales_count)
                VALUES (%s, %s, %s)
            """, (model_id, year, count))

# 변경 사항을 데이터베이스에 커밋
connection.commit()

# 커서와 연결 닫기
cursor.close()
connection.close()

In [ ]:
#지약별 코드 - 출력 성공 / 성별,용도 미출력
import urllib.request
import json
from bs4 import BeautifulSoup
import pymysql

# URL 설정
urls = [
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "개인",
        "gender": "남성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "개인",
        "gender": "여성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "개인",
        "gender": "남성",
        "country": "수입"
    }
    # 나머지 URL 생략...
]

# MySQL 데이터베이스 연결
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    database='pjt1',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connection.cursor()

# 테이블 생성 (지역별 브랜드 등록 차량 수 저장)
cursor.execute("DROP TABLE IF EXISTS region_car_sales")
create_region_car_sales_query = """
CREATE TABLE region_car_sales (
    id INT AUTO_INCREMENT PRIMARY KEY,
    region VARCHAR(255) NOT NULL,    -- 등록 지역
    brand VARCHAR(255) NOT NULL,     -- 제조사
    count INT NOT NULL,              -- 등록 차량 수
    year INT NOT NULL,                -- 등록 연도
    gender VARCHAR(10) NOT NULL,     -- 성별 (남성, 여성, 해당 없음)
    customer_type VARCHAR(10) NOT NULL  -- 고객 유형 (개인, 법인)
)
"""
cursor.execute(create_region_car_sales_query)

# 반복문을 통해 각 URL에서 데이터를 가져와 처리
for url_info in urls:
    url = url_info["url"]
    year = url_info["year"]
    country = url_info["country"]
    gender = url_info["gender"]
    country = url_info["country"]

    # HTML 요청 및 파싱
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    # 스크립트 태그에서 데이터를 포함한 텍스트 추출
    li_list = soup.select('body > script')
    text = li_list[5].text

    # JSON 데이터의 시작과 끝 인덱스를 찾아 슬라이싱
    start_index = text.find('{"MAP_DATA"')
    end_index = text.rfind(',"result":"OK"}') + len(',"result":"OK"}')
    map_data_text = text[start_index:end_index]

    # 불필요한 슬래시와 따옴표 제거
    map_data_text = map_data_text.replace("\\'", "'")

    # JSON 문자열을 파이썬 딕셔너리로 변환
    data = json.loads(map_data_text)

    # 데이터가 지역별로 등록된 차량 수임을 가정
    for item in data['MAP_DATA']:
        region = item['JUSO_SIDO']   # 지역
        brand = item['ORG_CAR_MAKER_KOR']  # 브랜드 이름
        count = item['CNT']          # 차량 등록 수

        # 지역별 브랜드 차량 등록 수 삽입
        insert_query = """
        INSERT INTO region_car_sales (region, brand, count, year)
        VALUES (%s, %s, %s, %s)
        """
        cursor.execute(insert_query, (region, brand, count, year))

# 변경 사항을 데이터베이스에 커밋
connection.commit()

# 커서와 연결 닫기
cursor.close()
connection.close()

In [ ]:
#지역별 데이터
import urllib.request
import json
from bs4 import BeautifulSoup
import pymysql

# URL 설정
# URL 설정
urls = [
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "개인",
        "gender": "남성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "개인",
        "gender": "여성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "개인",
        "gender": "남성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "개인",
        "gender": "여성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "개인",
        "gender": "남성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "개인",
        "gender": "여성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "개인",
        "gender": "남성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "개인",
        "gender": "여성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "수입"
    }

]

# MySQL 데이터베이스 연결
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    database='pjt1',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connection.cursor()

# 테이블 생성 (지역별 브랜드 등록 차량 수 저장, 성별과 고객유형 추가)
cursor.execute("DROP TABLE IF EXISTS region_car_sales")
create_table_query = """
CREATE TABLE region_car_sales (
    id INT AUTO_INCREMENT PRIMARY KEY,
    region VARCHAR(255) NOT NULL,    -- 등록 지역
    brand VARCHAR(255) NOT NULL,     -- 제조사
    count INT NOT NULL,              -- 등록 차량 수
    year INT NOT NULL,               -- 등록 연도
    gender VARCHAR(10) NOT NULL,     -- 성별 (남성, 여성, 해당 없음)
    customer_type VARCHAR(10) NOT NULL  -- 고객 유형 (개인, 법인)
)
"""
cursor.execute(create_table_query)

# 반복문을 통해 각 URL에서 데이터를 가져와 처리
for url_info in urls:
    url = url_info["url"]
    year = url_info["year"]
    customer_type = url_info["customer_type"]
    gender = url_info["gender"]
    country = url_info["country"]

    # HTML 요청 및 파싱
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    # 스크립트 태그에서 데이터를 포함한 텍스트 추출
    li_list = soup.select('body > script')
    text = li_list[5].text

    # JSON 데이터의 시작과 끝 인덱스를 찾아 슬라이싱
    start_index = text.find('{"MAP_DATA"')
    end_index = text.rfind(',"result":"OK"}') + len(',"result":"OK"}')
    map_data_text = text[start_index:end_index]

    # 불필요한 슬래시와 따옴표 제거
    map_data_text = map_data_text.replace("\\'", "'")

    # JSON 문자열을 파이썬 딕셔너리로 변환
    data = json.loads(map_data_text)

    # 데이터가 지역별로 등록된 차량 수임을 가정
    for item in data['MAP_DATA']:
        region = item['JUSO_SIDO']   # 지역
        brand = item['ORG_CAR_MAKER_KOR']  # 브랜드 이름
        count = item['CNT']          # 차량 등록 수

        # 지역별 브랜드 차량 등록 수 삽입 (성별과 고객유형 추가)
        insert_query = """
        INSERT INTO region_car_sales (region, brand, count, year, gender, customer_type)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (region, brand, count, year, gender, customer_type))

# 변경 사항을 데이터베이스에 커밋
connection.commit()

# 커서와 연결 닫기
cursor.close()
connection.close()

In [ ]:
#지역별 코드 완성
import urllib.request
import json
from bs4 import BeautifulSoup
import pymysql

# URL 설정
urls = [
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "개인",
        "gender": "남성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "개인",
        "gender": "여성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2023,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "개인",
        "gender": "남성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "개인",
        "gender": "여성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202301&TO_REGIST_DE=202312&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2023,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "개인",
        "gender": "남성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "개인",
        "gender": "여성",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EA%B5%AD%EC%82%B0",
        "year": 2024,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "국산"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%82%A8%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "개인",
        "gender": "남성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EC%97%AC%EC%84%B1&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "개인",
        "gender": "여성",
        "country": "수입"
    },
    {
        "url": "https://carcharts-free.carisyou.net/brand/url/brandKoreaMapChart?FROM_REGIST_DE=202401&TO_REGIST_DE=202407&SOU_GEN=%EB%B2%95%EC%9D%B8%EB%B0%8F%EC%82%AC%EC%97%85%EC%9E%90&CL_HMMD_IMP_SE_NM=%EC%88%98%EC%9E%85",
        "year": 2024,
        "customer_type": "법인",
        "gender": "해당 없음",
        "country": "수입"
    }

]

# MySQL 데이터베이스 연결
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    database='pjt1',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connection.cursor()

# 테이블 생성 (지역별 브랜드 등록 차량 수 저장)
cursor.execute("DROP TABLE IF EXISTS region_car_sales")
create_table_query = """
CREATE TABLE region_car_sales (
    id INT AUTO_INCREMENT PRIMARY KEY,
    region VARCHAR(255) NOT NULL,    -- 등록 지역
    brand VARCHAR(255) NOT NULL,     -- 제조사
    count INT NOT NULL,              -- 등록 차량 수
    year INT NOT NULL,               -- 등록 연도
    gender VARCHAR(10) NOT NULL,     -- 성별 (남성, 여성, 해당 없음)
    customer_type VARCHAR(10) NOT NULL  -- 고객 유형 (개인, 법인)
)
"""
cursor.execute(create_table_query)

# 반복문을 통해 각 URL에서 데이터를 가져와 처리
for url_info in urls:
    url = url_info["url"]
    year = url_info["year"]
    customer_type = url_info["customer_type"]
    gender = url_info["gender"]
    country = url_info["country"]

    # HTML 요청 및 파싱
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    # 스크립트 태그에서 데이터를 포함한 텍스트 추출
    li_list = soup.select('body > script')
    
    # 디버깅: 스크립트 리스트 출력
    print(f"li_list length: {len(li_list)}")
    
    # 스크립트에 필요한 데이터가 있는지 확인 (디버깅용)
    for index, script in enumerate(li_list):
        print(f"Script {index}: {script.text[:100]}...")  # 첫 100자를 출력해서 데이터 위치 확인
    
    # 우선 첫 번째 스크립트로 설정 (원래는 li_list[5]로 설정됨, 디버깅 필요)
    text = li_list[5].text if len(li_list) > 5 else li_list[-1].text

    # JSON 데이터의 시작과 끝 인덱스를 찾아 슬라이싱
    start_index = text.find('{"MAP_DATA"')
    end_index = text.rfind(',"result":"OK"}') + len(',"result":"OK"}')
    
    # 잘못된 경우 처리 (디버깅용)
    if start_index == -1 or end_index == -1:
        print(f"Error processing URL: {url}")
        continue  # 데이터를 찾을 수 없으면 다음 URL로 건너뜁니다.

    map_data_text = text[start_index:end_index]

    # 불필요한 슬래시와 따옴표 제거
    map_data_text = map_data_text.replace("\\'", "'")

    # JSON 문자열을 파이썬 딕셔너리로 변환
    try:
        data = json.loads(map_data_text)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from URL: {url} - {e}")
        continue  # JSON 파싱 실패 시 다음 URL로 건너뜁니다.

    # 데이터가 지역별로 등록된 차량 수임을 가정
    if 'MAP_DATA' not in data:
        print(f"MAP_DATA not found in data for URL: {url}")
        continue  # MAP_DATA가 없으면 건너뜁니다.

    for item in data['MAP_DATA']:
        region = item.get('JUSO_SIDO', 'Unknown')   # 지역
        brand = item.get('ORG_CAR_MAKER_KOR', 'Unknown')  # 브랜드 이름
        count = item.get('CNT', 0)          # 차량 등록 수

        # 지역별 브랜드 차량 등록 수 삽입
        insert_query = """
        INSERT INTO region_car_sales (region, brand, count, year, gender, customer_type)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (region, brand, count, year, gender, customer_type))

# 변경 사항을 데이터베이스에 커밋
connection.commit()

# 커서와 연결 닫기
cursor.close()
connection.close()